In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset

# Load protein sequences from a FASTA file
fasta_file = "/content/mus_train.fasta"
sequences = []
labels = []

with open(fasta_file, "r") as f:
    record_lines = []
    for line in f:
        if line.startswith(">"):
            if record_lines:
                sequences.append("".join(record_lines))
                record_lines = []
            labels.append(1 if "positive" in line else 0)  # Adjust labels accordingly
        else:
            record_lines.append(line.strip())
    if record_lines:
        sequences.append("".join(record_lines))

# Convert sequences and labels to tensors
data = {'sequence': sequences, 'label': labels}
df = pd.DataFrame(data)
'''
# Sample protein sequences in FASTA format
fasta_data = """
>positive
MVKVGVNGFGRIGRLVTRAAFNSGKVDVAVLDSGGTFIACSDKSAAVEAPQF
>positive
KFYNLCPMMEKEGKIALNDQKIVQLLREKTGEEKILVNLAARYFSYETSSA
>positive
MGDVEKGKKIFIMKCSQCHTVEKGGKHKTGPNEKGKKIFVQKCSQCHTVETG
>positive
MKTVRQERLKSIVRILERSKEPVSGAELRGKILRLQHPNIVRLGESNVDLV
>positive
MGIDYKAWRGELTQIKYTANPMFMLGPNIEGTTGEIVGTGGVNGEVTAWKG
>seq6
MADVLTLEDSWLSFYHLLFDGSDGKKQYAEELLDTLLRTEVAGYLTADLKS
>seq7
MQSNGFYLRAKAISTTAVGGVVEQSGVDVANLVGSAKILNTELLDTLVQQ
>seq8
MHGVPAKTFLFSTQPATAKKGVGGNNTFVRVVPQNKILSKHFSNAAAKKQ
>seq9
MATSLVNFSGELDTSAPVVRKPRKGKLMSSLGLPFSDVPVGPMGVGVSAI
>seq10
MKMTGMTLAGGFGYAAKANIVGNTPTSLRDAVKLADAVAAKGIKRVTYSK"""

# Convert FASTA data to a DataFrame
sequences = []
labels = []

for record in fasta_data.strip().split("\n"):
    if record.startswith(">"):
        labels.append(1 if "positive" in record else 0)  # Adjust labels accordingly
    else:
        sequences.append(record)

data = {'sequence': sequences, 'label': labels}
df = pd.DataFrame(data)
'''
# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Convert sequences and labels to tensors
sequences = df['sequence'].tolist()
labels = df['label'].tolist()


# Tokenize sequences and prepare data for BERT
encodings = tokenizer(sequences, truncation=True, padding=True, max_length=128)
input_ids = torch.tensor(encodings['input_ids'])
attention_mask = torch.tensor(encodings['attention_mask'])
labels = torch.tensor(labels)
print(labels)
# Define cross-validation
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize lists to store accuracy scores
accuracy_scores = []
test_label =[]
yscore= []
for fold, (train_idx, val_idx) in enumerate(skf.split(input_ids, labels)):
    print(f"Fold {fold+1}/{num_folds}")

    # Split data into training and validation sets
    train_input_ids = input_ids[train_idx]
    train_attention_mask = attention_mask[train_idx]
    train_labels = labels[train_idx]

    val_input_ids = input_ids[val_idx]
    val_attention_mask = attention_mask[val_idx]
    val_labels = labels[val_idx]

    train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
    val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_labels)

    # Define optimizer and training loop
    optimizer = AdamW(model.parameters(), lr=1e-5)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

    for epoch in range(3):  # adjust the number of epochs as needed
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            input_batch = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
            outputs = model(**input_batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

    # Evaluate on validation data
    model.eval()
    val_loader = DataLoader(val_dataset, batch_size=32)
    predictions = []
    with torch.no_grad():
        for batch in val_loader:
            input_batch = {'input_ids': batch[0], 'attention_mask': batch[1]}
            outputs = model(**input_batch)
            logits = outputs.logits
            test_label.append(logits)
            preds = torch.argmax(logits, dim=1)
            predictions.extend(preds.cpu().numpy())

    accuracy = accuracy_score(val_labels, predictions)
    accuracy_scores.append(accuracy)
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(test_label)
# Calculate and print average accuracy over folds
average_accuracy = np.mean(accuracy_scores)
print(f"Average Accuracy: {average_accuracy:.4f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([0, 0, 0,  ..., 0, 0, 0])
Fold 1/5


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
